In [0]:
# System_seed table info
dbutils.widgets.text("System_Id", "")
dbutils.widgets.text("tb_system_seed", "")

# Container Mounting credentials and info
dbutils.widgets.text("storage_account_name", "")
dbutils.widgets.text("storage_account_access_key", "")

In [0]:
# System_seed table info
System_Id = int(dbutils.widgets.get("System_Id"))
tb_system_seed = dbutils.widgets.get("tb_system_seed")

# Container Mounting credentials and info
storage_account_name = dbutils.widgets.get("storage_account_name")
storage_account_access_key = dbutils.widgets.get("storage_account_access_key")

print(System_Id)
print(storage_account_name)
print(storage_account_access_key)
print(tb_system_seed)

4
pracpipeline
8DmFEfAaT2VoAD2oFUhEeDOs6NQEsUNlyYjg6LzA5TYEcwQhTD+JEr5d5LwVDjZwMlD7/Pe8xH2P+AStwJ5tQA==
bcp_control.tb_system_seed


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException
from datetime import datetime
from pyspark.sql.functions import col
from pyspark.sql.functions import col, upper

system_id = System_Id
records_read=0
record_inserted=0
df=""
trigged_by = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()

In [0]:
spark = SparkSession.builder.getOrCreate()

try:
    # Execute the Spark SQL query
    file_info = spark.sql("""
        SELECT *
        FROM {}
        WHERE System_Id = {}
    """.format(tb_system_seed, System_Id)).first()

    if file_info is None:
        raise ValueError("File details not found in system_seed table for system_id = {}".format(System_Id))

    source_db_container_name = file_info.Source_DB
    source_object_name = file_info.Source_Object_Name
    target_db = file_info.Target_DB
    target_object_name = file_info.Target_Object_Name
    source_Type = file_info.Source_Type	
    target_Type = file_info.Target_Type
    isDelta = file_info.IsDelta
    delta_Field = file_info.Delta_Field

    print(source_db_container_name)
    print(source_object_name)
    print(target_db)
    print(target_object_name)
    print(isDelta)
    print(delta_Field)
    print(source_Type)
except (AnalysisException, ValueError, Exception) as error:
    print("Error occurred while processing system_seed input:")
    my_json={"records_read":records_read,"record_inserted":record_inserted,"trigged_by":trigged_by,"exception": str(error),"system_id":system_id}
    dbutils.notebook.exit(my_json)

integrated
Transaction_Table_Details
curated
Transaction_Table_Details
False
NA
delta


In [0]:
def source_mount(source_db_container_name,storage_account_name,storage_account_access_key):
    dbutils.fs.mount(
        source=f"wasbs://{source_db_container_name}@{storage_account_name}.blob.core.windows.net",
        mount_point=f"/mnt/{source_db_container_name}",
        extra_configs={
            f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_access_key
        },
    )
    print("Mount successful!")
try:
    source_mount(source_db_container_name,storage_account_name,storage_account_access_key)
except Exception as e:
    print(f"Already mounted {str(e).partition('is:')[0]}")

Already mounted An error occurred while calling o953.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/integrated; nested exception 


In [0]:
def source_mount(target_db,storage_account_name,storage_account_access_key):
    dbutils.fs.mount(
        source=f"wasbs://{target_db}@{storage_account_name}.blob.core.windows.net",
        mount_point=f"/mnt/{target_db}",
        extra_configs={
            f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_access_key
        },
    )
    print("Mount successful!")
try:
    source_mount(target_db,storage_account_name,storage_account_access_key)
except Exception as e:
    print(f"Already mounted {str(e).partition('is:')[0]}")

Already mounted An error occurred while calling o953.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/curated; nested exception 


In [0]:
def read_delta_file(source_db_container_name, source_object_name):
    if source_Type == "csv":
        print("csv")
        path = f"dbfs:/mnt/{source_db_container_name}/{source_object_name}.{source_Type}"
    else:
        print("delta")
        path = f"dbfs:/mnt/{source_db_container_name}/{source_object_name}"
        
    df = spark.read.format(source_Type).option("header", "true").option("inferSchema", "true").load(path)

    display(df.count())
    display(df)
    return df
try:
    df = read_delta_file(source_db_container_name, source_object_name)
    records_read=df.count()
except Exception as exp:
    exp=str(exp)
    my_json={"records_read":records_read,"record_inserted":record_inserted,"trigged_by":trigged_by,"exception":exp,"system_id":system_id}
    dbutils.notebook.exit(my_json)

delta
50

id,Member ID,Merchant ID,Transaction ID,Expr_acct_crd,Transaction Authentication date,Transaction Date,Original Amount of Transaction,Amount Transaction,Central Processing Date,Conversion Date,PS2000 Authentication Amount,Interchange Percentage rate,Interchange Percentage Intermediate rate,Authentication Cdt,Authentication_Ctm,Transaction Detail Indicator,Additional Data Indicator,Debit ACH indicator,Fraud Segment Number,Merchant Volume Indicator,Transaction Description,Merchant Flag,PS2000 Transaction ID,Source Decimal,Conversion Rate,Transaction Markup Amount,TLP_Type Transaction,Register Number,Receipt Number,TLP_Transaction Expiry Date,National Reimbursement Fee,Merchant DBA Name,Merchant DBA City,Merchant DBA State,Merchant DBA Country,Merchant Number,Pcas_Flag,cwb_Code,Floor limit code,Mail Phone Code,Authorisation Code,Trmnl_Capability,Card Holder ID Code,Reimbursement Attribute Code,ATM Account Select,Authorisation Character Indicator,Payment Service ID,Class Code,Debit Credit Indicator,Merchant Terminal Indicator,Interchange Qualify,Banknet Reference Number,PS2000 Authorization Currency,PS2000 Authorization Respondent Code,ISO Address Verification Result,Credit Account_Ved Term ID,Non Print Flag,Credit Activated Term ID,Excluded Transaction ID,International Fee indicator,Application ID,Application ID Suffix,Minimum due Snapshot,Account Dipute_Out_Number,Ptntl_frd_Transaction Flag,Statement Beginning Current Date and Time,Statement Date,Account Number,Account Number Original,Force Post Flag,Multiple Payment Reversal Indicator,Non-Sufficient Funds Cross-Cycle Indicator,Outstanding type,Post Date,Post Time,potnl_frd_move flag,Reference number,Reverse Indicator,Transaction Authorisation,Transaction Display Flag,Transaction Fraud Flag,Transaction memo Flag,Transaction Account Function,Transaction Source,Terminal Entry,Amount of Transaction in US Dollars,test
1.49E+11,298455992,5748168,1.49E+11,Activate,20-06-2023,20-06-2023,800.9873725,980.6900951,20-06-2023,20-06-2023,831.9393703,1.257929668,1.243545411,Fail,Mobile,B,Yes,Yes,4,Low,Online Purchase,B,1.95E+11,1.358310142,1.33402116,3.161405319,Type2,72,"['75340', '65583', '82194', '72557']",20-06-2023,2.191223936,Riggs Group,South Michael,WI,Argentina,Merchant14182,Yes,B,L2,Yes,Auth19740,Cap1,ID2,Attr2,Acc1,Char2,PS2,Class2,Debit,Terminal1,Q2,Ref68772,INR,Resp48363,Matched,CreditTerm72479,N,CreditTerm95669,ExclTxn87978,Yes,AppID99973,PInh6UC8Eq,77.17963198,Dispute0,Yes,06-07-2023 20:50,20-06-2023,Acc71996,AccOrig20548,Yes,No,Yes,Type2,20-06-2023,20-06-2023,No,Ref30819,No,Auth38619,Yes,No,No,Function1,Source1,Entry3,null,null
4.75E+11,222238145,7520312,4.75E+11,Expired,05-07-2023,05-07-2023,445.6898336,701.8382662,05-07-2023,05-07-2023,470.9749958,1.901066897,1.28784316,Fail,Mobile,A,No,No,9,Low,Mobile Payment,B,2.71E+11,1.596193632,1.970957542,3.899791365,Type3,78,"['63244', '60026', '15835', '39788']",05-07-2023,9.454337896,Smith-Bradley,Port Angela,NV,Sao Tome and Principe,Merchant66199,Yes,C,L3,No,Auth12644,Cap3,ID2,Attr1,Acc1,Char2,PS3,Class2,Debit,Terminal3,Q3,Ref81045,INR,Resp59634,Matched,CreditTerm17380,Y,CreditTerm88691,ExclTxn86178,Yes,AppID39191,MPTQjNIJGJ,66.49280204,Dispute1,No,06-07-2023 20:50,05-07-2023,Acc19326,AccOrig89356,No,Yes,Yes,Type1,05-07-2023,05-07-2023,No,Ref27568,Yes,Auth48926,No,No,Yes,Function1,Source3,Entry1,null,null
1.27E+11,115566789,3273159,1.27E+11,Activate,04-07-2023,04-07-2023,284.2505582,582.1825448,04-07-2023,04-07-2023,411.5087524,1.164641221,1.186334999,Fail,Web,B,Yes,No,7,Medium,Retail Store Purchase,B,6.27E+11,0.169976948,1.212949429,6.536199857,Type2,94,"['65254', '75552', '10673', '14536']",04-07-2023,1.724718089,Sullivan and Sons,Emmaton,PA,Sierra Leone,Merchant24016,Yes,C,L3,No,Auth41287,Cap3,ID3,Attr1,Acc2,Char2,PS1,Class3,Debit,Terminal2,Q2,Ref48994,RUB,Resp91856,Not Matched,CreditTerm33867,Y,CreditTerm51754,ExclTxn32505,No,AppID41386,TOrjAqS9HW,67.20968037,Dispute2,No,06-07-2023 20:50,04-07-2023,Acc75359,AccOrig28529,No,Yes,

In [0]:
from pyspark.sql.functions import col
duplicates_df = df.groupBy("id").count().where(col("count") > 1)
display(duplicates_df)

id,count
1.49E+11,2
1.78E+11,3
1.39E+11,2


In [0]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import concat_ws, substring, struct,StringType

# def create_id_column(row):
#     # Extract the last four digits from each column and concatenate them to form the 'id'
#     return f"{row['Member ID'][-4:]}{row['Social Security'][-4:]}{row['Access Number'][-4:]}"

# create_id_udf = udf(create_id_column, StringType())
# df_with_id = df.withColumn('id', create_id_udf(struct([df[col] for col in ['Member ID', 'Social Security', 'Access Number']])))

# display(df_with_id)

In [0]:
# from pyspark.sql import SparkSession
# from delta.tables import DeltaTable

# def get_record_inserted():
#     path = f"dbfs:/mnt/{target_db}/{target_object_name}"
#     delta_table_updates = DeltaTable.forPath(spark, path)
#     history_df = delta_table_updates.history()
#     history_df.createOrReplaceTempView("delta_history")
#     query = "SELECT * FROM delta_history ORDER BY version DESC LIMIT 1"
#     latest_history = spark.sql(query)
#     operation = latest_history.collect()[0].operation
#     if operation == "WRITE":
#         record_inserted = int(latest_history.collect()[0].operationMetrics["numOutputRows"])
#     else:
#         record_inserted = int(latest_history.collect()[0].operationMetrics["numTargetRowsInserted"])
#     return record_inserted
# # get_record_inserted()


from pyspark.sql import SparkSession
from delta.tables import DeltaTable

def get_record_inserted():
    path = f"dbfs:/mnt/{target_db}/{target_object_name}"
    try:
        spark = SparkSession.builder.appName("Get Record Inserted").getOrCreate()
        delta_table_updates = DeltaTable.forPath(spark, path)
        history_df = delta_table_updates.history()
        history_df.createOrReplaceTempView("delta_history")
        query = "SELECT * FROM delta_history ORDER BY version DESC LIMIT 1"
        latest_history = spark.sql(query)
        operation = latest_history.collect()[0].operation
        if operation == "WRITE":
            record_inserted = int(latest_history.collect()[0].operationMetrics["numOutputRows"])
        else:
            record_inserted = int(latest_history.collect()[0].operationMetrics["numTargetRowsInserted"])
        return record_inserted
    except Exception as e:
        print("An error occurred while getting record inserted:", e)
        return None



In [0]:
def check_path_exists(path):
    try:
        file_list = dbutils.fs.ls(path)
        return True
    except Exception as e:
        return False

In [0]:
def insert_data(data, path):
    spark.conf.set('spark.databricks.delta.schema.autoMerge.enabled',True)
    spark.conf.set("spark.databricks.delta.properties.defaults.columnMapping.mode","name")
    data.write.option("header","true").format("delta").option("overwriteSchema", "true").option("delta.columnMapping.mode", "name").mode('overwrite').save(path)

# try:
#     path = f"dbfs:/mnt/{target_db}/{target_object_name}"
#     insert_data(df, path)
# except Exception as exp:
#     exp=str(exp)
#     my_json={"records_read":records_read,"record_inserted":record_inserted,"trigged_by":trigged_by,"exception":exp,"system_id":system_id}
#     dbutils.notebook.exit(my_json)
# print(record_inserted)

In [0]:
from delta.tables import *

def incremental_load(path,sourceDF):
    targetDF = DeltaTable.forPath(spark, path)

    spark.conf.set('spark.databricks.delta.schema.autoMerge.enabled',True)
    targetDF.alias('target') \
    .merge(
        sourceDF.alias('source'),
        'target.id = source.id'
    ) \
    .whenMatchedUpdateAll() \
    .whenNotMatchedInsertAll()\
    .execute()

In [0]:
try:
    outputpath=f"dbfs:/mnt/{target_db}/{target_object_name}"
    if (isDelta == True):
        if (check_path_exists(outputpath)):
            print("exist")
            incremental_load(outputpath,df)
        else:
            print("doesnt exist")
            insert_data(df,outputpath)
    else:
        insert_data(df,outputpath)
except Exception as e:
    print(e)
    exp=str(e)
    if("PATH_NOT_FOUND" in exp):
         insert_data(df,outputpath)
    else:
        my_json={"records_read":records_read,"record_inserted":record_inserted,"trigged_by":trigged_by,"exception":exp,"system_id":system_id}
        dbutils.notebook.exit(my_json)

In [0]:
def read_curated_file(target_db, target_object_name):
    path = f"dbfs:/mnt/{target_db}/{target_object_name}"
    df = spark.read.format(target_Type).option("header", "true").load(path)
    display(df)
    display(df.count())
try:
    read_curated_file(target_db, target_object_name)
except Exception as exp:
    exp=str(exp)
    my_json={"records_read":records_read,"record_inserted":record_inserted,"trigged_by":trigged_by,"exception":exp,"system_id":system_id}
    dbutils.notebook.exit(my_json)

id,Member ID,Merchant ID,Transaction ID,Expr_acct_crd,Transaction Authentication date,Transaction Date,Original Amount of Transaction,Amount Transaction,Central Processing Date,Conversion Date,PS2000 Authentication Amount,Interchange Percentage rate,Interchange Percentage Intermediate rate,Authentication Cdt,Authentication_Ctm,Transaction Detail Indicator,Additional Data Indicator,Debit ACH indicator,Fraud Segment Number,Merchant Volume Indicator,Transaction Description,Merchant Flag,PS2000 Transaction ID,Source Decimal,Conversion Rate,Transaction Markup Amount,TLP_Type Transaction,Register Number,Receipt Number,TLP_Transaction Expiry Date,National Reimbursement Fee,Merchant DBA Name,Merchant DBA City,Merchant DBA State,Merchant DBA Country,Merchant Number,Pcas_Flag,cwb_Code,Floor limit code,Mail Phone Code,Authorisation Code,Trmnl_Capability,Card Holder ID Code,Reimbursement Attribute Code,ATM Account Select,Authorisation Character Indicator,Payment Service ID,Class Code,Debit Credit Indicator,Merchant Terminal Indicator,Interchange Qualify,Banknet Reference Number,PS2000 Authorization Currency,PS2000 Authorization Respondent Code,ISO Address Verification Result,Credit Account_Ved Term ID,Non Print Flag,Credit Activated Term ID,Excluded Transaction ID,International Fee indicator,Application ID,Application ID Suffix,Minimum due Snapshot,Account Dipute_Out_Number,Ptntl_frd_Transaction Flag,Statement Beginning Current Date and Time,Statement Date,Account Number,Account Number Original,Force Post Flag,Multiple Payment Reversal Indicator,Non-Sufficient Funds Cross-Cycle Indicator,Outstanding type,Post Date,Post Time,potnl_frd_move flag,Reference number,Reverse Indicator,Transaction Authorisation,Transaction Display Flag,Transaction Fraud Flag,Transaction memo Flag,Transaction Account Function,Transaction Source,Terminal Entry,Amount of Transaction in US Dollars,test
1.49E+11,298455992,5748168,1.49E+11,Activate,20-06-2023,20-06-2023,800.9873725,980.6900951,20-06-2023,20-06-2023,831.9393703,1.257929668,1.243545411,Fail,Mobile,B,Yes,Yes,4,Low,Online Purchase,B,1.95E+11,1.358310142,1.33402116,3.161405319,Type2,72,"['75340', '65583', '82194', '72557']",20-06-2023,2.191223936,Riggs Group,South Michael,WI,Argentina,Merchant14182,Yes,B,L2,Yes,Auth19740,Cap1,ID2,Attr2,Acc1,Char2,PS2,Class2,Debit,Terminal1,Q2,Ref68772,INR,Resp48363,Matched,CreditTerm72479,N,CreditTerm95669,ExclTxn87978,Yes,AppID99973,PInh6UC8Eq,77.17963198,Dispute0,Yes,06-07-2023 20:50,20-06-2023,Acc71996,AccOrig20548,Yes,No,Yes,Type2,20-06-2023,20-06-2023,No,Ref30819,No,Auth38619,Yes,No,No,Function1,Source1,Entry3,null,null
4.75E+11,222238145,7520312,4.75E+11,Expired,05-07-2023,05-07-2023,445.6898336,701.8382662,05-07-2023,05-07-2023,470.9749958,1.901066897,1.28784316,Fail,Mobile,A,No,No,9,Low,Mobile Payment,B,2.71E+11,1.596193632,1.970957542,3.899791365,Type3,78,"['63244', '60026', '15835', '39788']",05-07-2023,9.454337896,Smith-Bradley,Port Angela,NV,Sao Tome and Principe,Merchant66199,Yes,C,L3,No,Auth12644,Cap3,ID2,Attr1,Acc1,Char2,PS3,Class2,Debit,Terminal3,Q3,Ref81045,INR,Resp59634,Matched,CreditTerm17380,Y,CreditTerm88691,ExclTxn86178,Yes,AppID39191,MPTQjNIJGJ,66.49280204,Dispute1,No,06-07-2023 20:50,05-07-2023,Acc19326,AccOrig89356,No,Yes,Yes,Type1,05-07-2023,05-07-2023,No,Ref27568,Yes,Auth48926,No,No,Yes,Function1,Source3,Entry1,null,null
1.27E+11,115566789,3273159,1.27E+11,Activate,04-07-2023,04-07-2023,284.2505582,582.1825448,04-07-2023,04-07-2023,411.5087524,1.164641221,1.186334999,Fail,Web,B,Yes,No,7,Medium,Retail Store Purchase,B,6.27E+11,0.169976948,1.212949429,6.536199857,Type2,94,"['65254', '75552', '10673', '14536']",04-07-2023,1.724718089,Sullivan and Sons,Emmaton,PA,Sierra Leone,Merchant24016,Yes,C,L3,No,Auth41287,Cap3,ID3,Attr1,Acc2,Char2,PS1,Class3,Debit,Terminal2,Q2,Ref48994,RUB,Resp91856,Not Matched,CreditTerm33867,Y,CreditTerm51754,ExclTxn32505,No,AppID41386,TOrjAqS9HW,67.20968037,Dispute2,No,06-07-2023 20:50,04-07-2023,Acc75359,AccOrig28529,No,Yes,

50

In [0]:
my_json={"records_read":records_read,"record_inserted":get_record_inserted(),"trigged_by":trigged_by,"exception":'',"system_id":system_id}
dbutils.notebook.exit(my_json)